In [ ]:
import argparse
import os
import numpy as np
import math

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

os.environ["CUDA_VISIBLE_DEVICES"] = "1,2,3,4,5,6,7,8"


os.makedirs("images", exist_ok=True)

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument("--n_epochs", type=int, default=200, help="number of epochs of training")
parser.add_argument("--batch_size", type=int, default=30, help="size of the batches")
parser.add_argument("--lr", type=float, default=0.0002, help="adam: learning rate")
parser.add_argument("--b1", type=float, default=0.5, help="adam: decay of first order momentum of gradient")
parser.add_argument("--b2", type=float, default=0.999, help="adam: decay of second order momentum of gradient")
parser.add_argument("--n_cpu", type=int, default=8, help="number of cpu threads to use during batch generation")
parser.add_argument("--latent_dim", type=int, default=100, help="dimensionality of the latent space")
parser.add_argument("--img_size", type=int, default=28, help="size of each image dimension")
parser.add_argument("--channels", type=int, default=1, help="number of image channels")
parser.add_argument("--sample_interval", type=int, default=400, help="interval between image samples")
opt = parser.parse_args()
print(opt) #　打印初始参数

In [ ]:
img_shape = (opt.channels, opt.img_size, opt.img_size) #img_shape: (1, 28, 28)

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(opt.latent_dim, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, int(np.prod(img_shape))),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.size(0), *img_shape)
        return img



In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(int(np.prod(img_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid(),
        )

    def forward(self, img):
        img_flat = img.view(img.size(0), -1)
        validity = self.model(img_flat)

        return validity



In [ ]:
os.makedirs("../../data/mnist", exist_ok=True)

dataloader = torch.utils.data.DataLoader(
    datasets.MNIST(
        "../../data/mnist",
        train=True,
        download=True,
        transform=transforms.Compose(
            [transforms.Resize(opt.img_size), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
        ),
    ),
    batch_size=opt.batch_size,
    shuffle=True,
)

In [ ]:
adversarial_loss = torch.nn.BCELoss(reduction='mean')


generator = Generator()
discriminator = Discriminator()


In [ ]:
CUDA = True if torch.cuda.is_available() else False
if CUDA:
    # 单个GPU，可行
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()

    # 单个GPU，可行
    # DEVICE = torch.device("cuda" if CUDA else "cpu")
    # generator = generator.to(DEVICE)
    # discriminator = discriminator.to(DEVICE)
    # adversarial_loss = adversarial_loss.to(DEVICE)

    # 尝试多GPU，试了一天，没成功
    # DEVICE = torch.device("cuda:0" if CUDA else "cpu")
    # generator = nn.DataParallel(generator, device_ids=[0, 1, 2, 3, 4, 5, 6, 7])
    # generator = generator.to(DEVICE)
    # discriminator = nn.DataParallel(discriminator, device_ids=[0, 1, 2, 3, 4, 5, 6, 7])
    # discriminator = discriminator.to(DEVICE)
    # adversarial_loss = nn.DataParallel(adversarial_loss, device_ids=[0, 1, 2, 3, 4, 5, 6, 7])
    # adversarial_loss = adversarial_loss.to(DEVICE)


In [ ]:
optimizer_G = torch.optim.Adam(generator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))

In [ ]:
Tensor = torch.cuda.FloatTensor if CUDA else torch.FloatTensor
for epoch in range(opt.n_epochs): # n_epochs: 200
    for i, (imgs, _) in enumerate(dataloader):        
        # valid设为１，fake设为０
        valid = Variable(Tensor(imgs.size(0), 1).fill_(1.0), requires_grad=False)
        fake = Variable(Tensor(imgs.size(0), 1).fill_(0.0), requires_grad=False)

        # 真实图像
        real_imgs = Variable(imgs.type(Tensor))

        # -----------------
        #  训练生成器
        # -----------------

        optimizer_G.zero_grad()

        # 以噪声作为生成器的输入
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], opt.latent_dim))))

        # 生成图像
        gen_imgs = generator(z)

         # 生成器欺骗判别器的能力由g_loss表示
        g_loss = adversarial_loss(discriminator(gen_imgs), valid)

        # 反向传播　　　　　
        g_loss.backward(torch.ones_like(g_loss))

        # 更新优化器
        optimizer_G.step()

        # ---------------------
        #  训练判别器
        # ---------------------

        optimizer_D.zero_grad()

        # 评价判别器将生成图片归类为真或假的能力由real_loss和fake_loss表示
        real_loss = adversarial_loss(discriminator(real_imgs), valid)
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
        # 两者取平均得到d_loss
        d_loss = (real_loss + fake_loss) / 2

        # 反向传播
        d_loss.backward(torch.ones_like(d_loss))
        optimizer_D.step()

        print(
            "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
            % (epoch, opt.n_epochs, i, len(dataloader), d_loss.item(), g_loss.item())
        )
   
        # 当前循环的总次数
        batches_done = epoch * len(dataloader) + i
        # 按一定的间隔输出生成的图像
        if batches_done % opt.sample_interval == 0:
            save_image(gen_imgs.data[:25], "images/%d.png" % batches_done, nrow=5, normalize=True)




